# Задание к модулю "RL в ИБ"
Есть CSV-файл c ежедневным количеством задач, которое должны обработать аналитики SOC. Каждый день можно вызывать на работу разное количество аналитиков. Если их окажется меньше, чем задач - это плохо, потому что не все задачи будут обработаны. Если больше - тоже плохо, потому что придётся заплатить им больше, чем можно было бы. Ваша задача - с применением методов RL разработать модель, которая будет предсказывать оптимальные действия на каждый день - "вызывать" или "отпускать" аналитиков.

Среда, в которой будет действовать модель, уже описана. Также подготовлена "глупая и медленная" модель, которую (как минимум) нужно будет превзойти.

Студенту предлагается реализовать метод обучения агента и его архитектуру (задания отмечены #TODO). Один из вариантов порядка работы алгоритма описан в виде комментариев на русском языке. Можно их не придерживаться и реализовать свой вариант модели RL.

## Импорт модулей, загрузка данных

In [ ]:
import numpy as np # линейная алгебра
import pandas as pd # обработка данных, чтение-запись
import time # оценка времени
import copy # дублирование ссылочных объектов
import chainer # нейронные сети
import chainer.functions
import chainer.links
from plotly.offline import init_notebook_mode, iplot, iplot_mpl # использование JavaScript в Jupyter-ноутбуке
init_notebook_mode()
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Если какого-то модуля не хватает:
#!pip install <module>

In [ ]:
data = pd.read_csv('soc_workers.csv')
data['Date'] = pd.to_datetime(data['Date']) # преобразование типа
data = data.set_index('Date') # индексация массива будет по полю "дата"
print(data.index.min(), data.index.max()) # определение границ временного интервала
data.head() # первые 5 строк для визуальной оценки

In [ ]:
date_split = '2016-01-01' # до этого дня - тренировочная выборка, после - тестовая
train = data[:date_split] # тренировочные данные
test = data[date_split:] # тестовые данные
len(train), len(test) # посмотрим, сколько данных в каждой выборке

In [ ]:
fig = make_subplots()
fig.append_trace(go.Scatter(x=train.index, y=train['Quantity'], mode='lines', name='train', line_color='blue'), row=1, col=1)
fig.append_trace(go.Scatter(x=test.index, y=test['Quantity'], mode='lines', name='test', line_color='red'), row=1, col=1)
fig.add_vline(x=date_split)
fig.show()

## Описание среды для работы агентов

In [ ]:
class Environment:
    """
    Среда для обучения агента повторению графика
    """
    def __init__(self, data, history_t=90):
        """
        Создание среды обучения из исходных данных. Среда возвращается очищенная
        Parameters
            data: список чисел, каждое из которых - потребность в аналитиках
            history_t: количество позиций, которое среда возвращает в виде состояния
        """
        self.data = data
        self.history_t = history_t # память среды в количестве позиций назад
        self.one_step_move = 10 # смещение при шаге (найм или отпускание аналитиков)
        self.reset() # сброс состояния среды
        
    def reset(self):
        """
        Сброс данных окружения
        Returns:
            obs: наблюдаемое состояние
        """
        self.t = 0
        self.done = False # показатель доигранности эпизода
        self.workers = self.data.iloc[0, :]['Quantity'] # количество нанятых аналитиков в начальный момент идеально
        self.history = [0 for _ in range(self.history_t)] # история изменений цен акций за заданный промежуток времени
        obs = [self.workers] + self.history # состояние
        return obs
    
    def step(self, act):
        """
        Переход из текущего состояние в следующее заданным действием
        Parameters
            act: выбранное действие: 0: stay, 1: buy, 2: sell
        Returns
            obs: новое состояние
            reward: полученная за действие награда
            done: показатель завершения эпизода
        """
        reward = 0 # награда за переход
        
        if act == 1: # действие - нанимать аналитиков
            self.workers += self.one_step_move
        elif act == 2: # действие - отпускать аналитиков
            self.workers = abs(self.workers - self.one_step_move) # без отпускания в минус 
        
        quantity_t = self.data.iloc[self.t, :]['Quantity'] # потребность в аналитиках на этом шаге
        quantity_pre_t = self.data.iloc[self.t - 1, :]['Quantity'] # потребность в аналитиках на предыдущем шаге
        reward = -abs(quantity_t - self.workers) # отличие требуемого количества аналитиков от реального
        
        self.t += 1 # переход к следующему временному отрезку
        self.history.pop(0) # удаление самого старого элемента истории
        # self.history.append(quantity_t - quantity_pre_t) # запись в историю изменения потребности в аналитиках за день
        self.history.append(quantity_t) # запись в историю потребности в аналитиках за день
        
        obs = [self.workers] + self.history # состояние, в которое перешёл агент
        return obs, reward, self.done

In [ ]:
env = Environment(train, history_t=90) # создаём новую среду с тренировочными данными
print(env.reset()) # сбрасываем состояние среды и смотрим обновлённое состояние
for _ in range(3):
    print(env.step(np.random.randint(3))) # делаем случайное действие

## Создание и обучение агента

### Описание архитектуры сети

In [ ]:
class Q_Network(chainer.Chain):
    """
    Архитектура модели 
    """
    def __init__(self, input_size, hidden_size, output_size):
        """
        Инициализация модели
        Parameters
            input_size: количество нейронов во входном слое (количество измерений состояния)
            hidden_size: количество нейронов в скрытом слое
            output_size: количество выходных нейронов (количество состояний)
        """
        super(Q_Network, self).__init__(
            #TODO
            raise NotImplementedError
        )

    def __call__(self, x):
        """
        Предсказание возможных наград
        Parameters
            x: текущее состояние
        Returns
            y: возможные награды за каждое действие из этого состояния
        """
        #TODO
        raise NotImplementedError
        return y

    def reset(self):
        """
        Очистка градиентов
        """
        #TODO
        raise NotImplementedError

### Описание метода обучения агента

In [ ]:
# DQN - Deep Q-Learning

def train_dqn(env, Q, epoch_num=50):
    """
    Обучение агента Deep Q-Learning
    Parameters
        env: окружение, в котором агент будет обучаться
        Q: архитектура модели
        epoch_num: количество эпох, которые следует учить агента, default=50
    Returns
        Q: обученная модель
        total_losses: потери за каждый эпизод
        total_rewards: награды за каждый эпизод
    """

    Q_ast = copy.deepcopy(Q) # "старый" агент (предыдущее состояние таблички)
    optimizer = chainer.optimizers.Adam() # выбор оптимизатора - Adam (для изменения скорости обучения)
    optimizer.setup(Q) # добавление модели для отслеживания оптимизатором

    epoch_num = epoch_num # количество эпох обучения агента
    step_max = len(env.data) - 1 # максимальное количество шагов в одному эпизоде
    memory_size = 200 # максимальный размер памяти
    batch_size = 20 # количество эпизодов в одном батче
    epsilon = 1.0 # вероятность, с которой будет выбираться случайное действие
    epsilon_decrease = 1e-3 # значение, на которое каждый шаг уменьшаеття вероятность выбора каждого действия
    epsilon_min = 0.1 # минимальное значение вероятности выбрать случайное действие
    start_reduce_epsilon = 200 # номер шага, после которых надо уменьшать вероятность выбора случайного действия
    train_freq = 10 # на каком шаге учиться (сколько шагов пропускать для накопления информации)
    update_q_freq = 20 # частота обновления модели (Q-таблицы)
    gamma = 0.97 # коэффициент дисконтирования перспективных (возможных) наград
    show_log_freq = 5 # частота отображения информации об обучении модели для пользователя

    memory = [] # память шагов
    total_step = 0 # общее количество сделанных шагов
    total_rewards = [] # история наград за каждый эпизод
    total_losses = [] # история потерь за каждый эпизод

    start = time.time() # время запуска обучения
    print('\t'.join(map(str, ['epoch', 'epsilon', 'total_step', 'log_reward', 'log_loss', 'elapsed_time'])))
    for epoch in range(epoch_num): # для каждой эпохи
        #TODO
        raise NotImplementedError
        
        # сбрасываем состояние окружения
        step = 0 # номер шага в эпохе
        done = False # критерий завершения
        total_reward = 0 # общая награда за эпоху
        total_loss = 0 # общее значние функции потерь за эпоху

        while not done and step < step_max: # пока эпизод не доигран и не превышено максимальное число шагов
            
            # вероятностно выбираем или случайное действие, или оптимальное (из текущей позиции)
            #TODO
            obs, reward, done = env.step(action) # агент делает действие и получает ответ от среды

            # сохраняем в память новое состояние, действие, старое состояние, показатель завершения эпизода
            memory.append((pobs, pact, reward, obs, done))
            # если память заполнена, удаляем самый старый элемент
            #TODO
            
            # если наиграли достаточно шагов, чтобы память была заполнена
                # если накопили достаточно шагов, чтобы учиться
                    # перестановка в случайном порядке
                    # перечисление от 0 до объёма памяти
                    # делим на группы (батчи) всю перемешанную память и для каждого
                        # создание массива из батча
                        # состояния
                        # действие
                        # награды
                        # old состояние
                        # показатели завершённости

                        q = Q(<состояния>) # предсказание возможных наград за каждый элемент из батча новой моделью
                        # предсказание максимальных наград старой моделью
                        # предсказания - преобразование к ndarray и копирование значений
                        # для каждого элемента в рассматриваемом батче
                            # награда на этом шаге + дисконтированная будущая награда (если не конец эпизода)
                        # сброс состояния
                        # функция ошибки как СКО вероятных наград, предсказанных новой и старой моделями
                        # добавляем значение функции ошибки батча к общему
                        # обратный проход нейронной сети с заданной ошибкой
                        # переход к следующему шагу оптимизатора

                # если пора менять Q-таблицу
                # старая модель "догоняет" новую

            # если вероятность выбора случайного действия больше минимальной и пора её уменьшать (первые шаги сделаны)
                # уменьшаем вероятность выбора случайного действия на заданное значение

            # награду за шаг прибавляем к общей награде эпизода
            # переходим к следующему состоянию (следующее состояние становится текущим)
            # увеличиваем количество шагов в этом эпизоде
            # увеличиваем общее количество шагов 

        # запоминаем награду за эпизод в историю
        # запоминаем значение суммы функции ошибки за эпизод в историю

        if (epoch + 1) % show_log_freq == 0: # если пора логировать успехи
            log_reward = sum(total_rewards[((epoch + 1) - show_log_freq):]) / show_log_freq # сумма наград за последние шаги
            log_loss = sum(total_losses[((epoch + 1) - show_log_freq):]) / show_log_freq # сумма потерь за последние шаги
            elapsed_time = time.time() - start # время, прошедшее с последней записи в лог
            print('\t'.join(map(str, [epoch + 1, epsilon, total_step, log_reward, log_loss, elapsed_time]))) # лог в консоль
            start = time.time() # обновляем значение времени
            
    return Q, total_losses, total_rewards # модель, потери и награды за каждый эпизод
    

### Создание агента (выбор гиперпараметров)

In [ ]:
# инициализация модели
#TODO
raise NotImplementedError
Q = Q_Network(params)

### Обучение агента (длительная операция)

In [ ]:
# обучение модели
Q, total_losses, total_rewards = train_dqn(Environment(train), Q, epoch_num=100)

### Оценка результатов обучения

In [ ]:
def plot_loss_reward(total_losses, total_rewards):
    """
    Отображение графиков обучения
    Parameters
        total_losses: потери за каждый эпизод
        total_rewards: награды за каждый эпизод
    """
    figure = make_subplots (rows=1, cols=2, subplot_titles=('loss', 'reward'), print_grid=False) # два окна графиков
    figure.append_trace(go.Scatter(y=total_losses, mode='lines', line=dict(color='skyblue')), 1, 1) # потери по эпизодам
    figure.append_trace(go.Scatter(y=total_rewards, mode='lines', line=dict(color='orange')), 1, 2) # награды по эпизодам
    figure['layout']['xaxis1'].update(title='epoch') # подпись горизонтальный оси
    figure['layout']['xaxis2'].update(title='epoch')
    figure['layout'].update(height=400, width=900, showlegend=False) # установка размеров графика
    iplot(figure) # рисование графика с применением JavaScript
    

In [ ]:
plot_loss_reward(total_losses, total_rewards)

## "Настоящая" работа агента: отпустим его зарабатывать деньги

Работа с использованием обученного агента

In [ ]:
def run_model(env, Q):
    """
    Применение обученной модели в среде
    Parameters:
        env: окружение, в котором будет работать агент
        Q: обученная модель
    Returns:
        workers_history: история реального количества работников каждый день
        rewards: сумма наград (правильных решений) за все эпизоды
        acts: история всех действий агента
    """
    workers_history = []
    pobs = env.reset() # сброс параметров среды
    acts = [] # действия агента за все эпизоды
    rewards = [] # награды агнента за все эпизоды
    for _ in range(len(env.data) - 1): # для каждого известного эпизода в данных
        pact = Q(np.array(pobs, dtype=np.float32).reshape(1, -1)) # вероятные награды для каждого действия
        pact = np.argmax(pact.data) # оптимальное действие по максимуму вероятной награды
        acts.append(pact) # запоминаем оптимальное действие
        obs, reward, done = env.step(pact) # переход к следующему состоянию
        rewards.append(reward) # запоминаем награды
        pobs = obs # следующее состояние становится текущим
        workers_history.append(obs[0]) # запоминаем реальное количество работников
        
    return workers_history, rewards, acts # заработок, награды, действия агента

Работа с использованием детерминированного "догоняющего" агента

In [ ]:
def run_stupid_slow_model(env):
    """
    Простая детерминированная модель. Работает "вдогонку" - сравнивает текущее значение
        с устаревшим на 30 шагов (месяц) и принимает решение
    Parameters
        env: окружение, в котором будет работать агент
    Returns:
        workers_history: история реального количества работников каждый день
        rewards: сумма наград (правильных решений) за все эпизоды
        acts: история всех действий агента
    """
    workers_history = []
    pobs = env.reset() # сброс параметров среды
    acts = [] # действия агента за все эпизоды
    rewards = [] # награды агнента за все эпизоды
    for _ in range(len(env.data) - 1): # для каждого известного эпизода в данных
        if pobs[0] < pobs[-30]: # если был недостаток
            pact = 1 # нанимать
        else: # если были лишние
            pact = 2 # отпускать
        acts.append(pact) # запоминаем оптимальное действие
        obs, reward, done = env.step(pact) # переход к следующему состоянию
        rewards.append(reward) # запоминаем награды
        pobs = obs # следующее состояние становится текущим
        workers_history.append(obs[0]) # запоминаем реальное количество работников
        
    return workers_history, rewards, acts # заработок, награды, действия агента

### Описание сред (работа на известных данных и на тех, которых он раньше не видел)

In [ ]:
train_env = Environment(train)
test_env = Environment(test)

### Само применение

In [ ]:
# известные данные
stupid_train_workers_history, stupid_train_rewards, stupid_train_acts = run_stupid_slow_model(train_env)
rl_train_workers_history, rl_train_rewards, rl_train_acts = run_model(train_env, Q)
# неизвестные данные
stupid_test_workers_history, stupid_test_rewards, stupid_test_acts = run_stupid_slow_model(test_env)
rl_test_workers_history, rl_test_rewards, rl_test_acts = run_model(test_env, Q)

### Рисование графиков

Отображение данных на графике

In [ ]:
fig = make_subplots()
print('Сумма наград за все тренировочные эпизоды:\n\tRL', int(sum(rl_train_rewards)),
      '\n\tstupid:', int(sum(stupid_train_rewards)))
print('Сумма наград за все тренировочные эпизоды:\n\tRL', int(sum(rl_test_rewards)),
      '\n\tstupid:', int(sum(stupid_test_rewards)))

fig.append_trace(go.Scatter(x=train.index, y=train['Quantity'], mode='lines', name='Потребность', line_color='green'), row=1, col=1)
fig.append_trace(go.Scatter(x=train.index, y=stupid_train_workers_history, mode='lines', name='Наличие (stupid)', line_color='black'), row=1, col=1)
fig.append_trace(go.Scatter(x=train.index, y=rl_train_workers_history, mode='lines', name='Наличие (RL)', line_color='red'), row=1, col=1)

fig.append_trace(go.Scatter(x=test.index, y=test['Quantity'], mode='lines', name='Потребность', line_color='green'), row=1, col=1)
fig.append_trace(go.Scatter(x=test.index, y=stupid_test_workers_history, mode='lines', name='Наличие (stupid)', line_color='black'), row=1, col=1)
fig.append_trace(go.Scatter(x=test.index, y=rl_test_workers_history, mode='lines', name='Наличие (RL)', line_color='red'), row=1, col=1)

fig.add_vline(x=date_split)
fig.show()